In [1]:
# https://github.com/openai/openai-cookbook/blob/main/examples/How_to_format_inputs_to_ChatGPT_models.ipynb
# https://stackoverflow.com/questions/77285102/how-to-format-a-few-shot-prompt-for-gpt4-chat-completion-api

In [2]:
# pip install --upgrade openai

In [3]:
import sys, os
p = os.path.abspath('..')
# p = p+r'\config'
sys.path.insert(1, p)


In [4]:
import warnings


import pandas as pd
import numpy as np
from tqdm import tqdm
import time
import config.config as conf

import json
from itertools import chain
warnings.filterwarnings("ignore", category=DeprecationWarning)
# https://wikidocs.net/233348

In [5]:
# import the OpenAI Python library for calling the OpenAI API
from openai import OpenAI
import os

client = OpenAI(api_key= conf.OEPN_AI_KEY)

In [6]:
# # Example OpenAI Python library request
# MODEL = "gpt-4o-2024-08-06"
# response = client.chat.completions.create(
#     model=MODEL,
#     messages=[
#         {"role": "system", "content": "You are a helpful assistant."},
#         {"role": "user", "content": "Explain asynchronous programming in the style of the pirate Blackbeard."},
#     ],
#     temperature=0,
# )

In [7]:
# print(json.dumps(json.loads(response.model_dump_json()), indent=4))

In [8]:
df = pd.read_csv('../../data/part_one_q_output_code_y.csv', index_col = 0)

In [9]:
df['question'][0]

'<Title>Why joblib is not recommended when saving keras model?</Title>. <Question><p>According to <a href="https://keras.io/getting_started/faq/#how-can-i-save-a-keras-model" rel="nofollow noreferrer">this</a> keras documentation, pickle is not recommended to save keras mode, and since <code>joblib.dump()</code> and <code>joblib.load()</code> are based on the Python pickle serialization model, joblib is also not recommended to save keras model. What is the reason for that ?</p>\n</Question>'

In [10]:
sys_promt="""\nYou are an expert in analyzing and categorizing the difficulty level of Python-related programming questions based on examples below. 
Questions are sourced from the "StackOverflow" community using the keyword "Python."\n"""

In [11]:
user_prompt_front = """\nFor the given (target) post that is marked by
    <target_post> </target_post>, please let me know the
    "Difficulty Level" of the target post, where the
    "Difficulty Level" can be one of the followings: (0) Basic, (1)
    Intermediate, and (2) Advanced\n"""

In [12]:
user_prompt_back = """\nplease output only the difficulty label of the target post, where it can only be one of, (0) Basic, (1)
    Intermediate, and (2) Advanced, 
    and mark '0' if it is basic, '1' if is is intermediate, and '2' if it is advanced question. 
    Please refer to the samples below that may be helpful to measure the baseline of the difficulty labels, where each
    of the sample is delimited by "----"\n
    """

In [13]:
diff_dict = {'Difficulty class : Basic':0 ,
 'Difficulty class : Intermediate':1, 
 'Difficulty class : Advanced':2}

In [14]:
df['answer_encode'] = df['answer'].apply(lambda x : diff_dict[x])

In [15]:
df.head()

,id,question,answer,answer_encode
0,74693871,<Title>Why joblib is not recommended when savi...,Difficulty class : Advanced,2
1,74751254,<Title>Removing all duplicate images with diff...,Difficulty class : Intermediate,1
2,74760874,<Title>How do I extract a binary pattern from ...,Difficulty class : Intermediate,1
3,74774389,<Title>How to join these 2 tables by date with...,Difficulty class : Intermediate,1
4,74778102,<Title>Error when trying to save a form in dja...,Difficulty class : Intermediate,1


In [16]:
diff_idx = {x : list(df[df['answer']==x].index) for x in list(diff_dict.keys())}

In [17]:
diff_s_idx = {}
for key, value in diff_idx.items():
    print(key, value)
    dic_col = f'{key}_sample_idx'
    diff_population = value
    diff_s_idx[dic_col] = np.random.choice(diff_population, size=3, replace=False)

Difficulty class : Basic [10, 21, 23, 25, 31, 35, 36, 38, 40]
Difficulty class : Intermediate [1, 2, 3, 4, 5, 6, 7, 8, 11, 12, 13, 14, 15, 16, 17, 18, 19, 22, 24, 26, 28, 29, 30, 32, 33, 37, 39]
Difficulty class : Advanced [0, 9, 20, 27, 34, 41, 42]


In [18]:
diff_s_idx

{'Difficulty class : Basic_sample_idx': array([23, 31, 35]),
 'Difficulty class : Intermediate_sample_idx': array([12, 24, 30]),
 'Difficulty class : Advanced_sample_idx': array([20, 27, 34])}

In [19]:
fewshot_q_id = list(chain.from_iterable(diff_s_idx.values()))


In [20]:
eval_q_id =np.setdiff1d(list(df.index), fewshot_q_id)

In [21]:
examples = []
for idx in fewshot_q_id:
# for idx in [19]:
    temp_dict = {"question" : str(df.loc[idx, 'question']),
                 "answer"   : str(df.loc[idx, 'answer'])}
    examples.append(temp_dict)

In [22]:
examples

[{'question': '<Title>How to convert values of tuple from a dictionary into a list?</Title>. <Question><p>I am trying to convert the values from a dictionary into a list.</p>\n<p><code>D = {&quotA&quot: [(1, 2)], &quotB&quot: [(3, 4), (5, 6)]}</code></p>\n<p>How do I get the following result?</p>\n<p><code>[[1, 2], [3,4], [5, 6]]</code></p>\n<p>I am trying to use max 1 for-loop</p>\n<pre><code># My attempt\nmy_list = list(D.values())\nprint(my_list)\n# [[(1, 2)], [(3, 4), (5, 6)]]\n</code></pre>\n</Question>',
  'answer': 'Difficulty class : Basic'},
 {'question': "<Title>How do I iterate for all HDF5 files and save them as csv files</Title>. <Question><p>I am writing a python code that will loop over my SMAP HDF5 (10,000) files. I want to extract soil moisture rootzone and vegetation greenness. My code looks like this</p>\n<pre><code>import os\nimport tables\nimport h5py\nimport datetime as dt\nimport glob\nimport h5py\nimport matplotlib.pyplot as mpl\nimport numpy as np\nimport os\ni

In [23]:
q_prompt = """"""
a=1
for q_a in examples : 
    q_prompt = q_prompt+"""\n----\n"""
    q_prompt = q_prompt+f"{q_a['answer']}\n"
    q_prompt = q_prompt+"""<post>\n"""
    q_prompt = q_prompt+f"{q_a['question']}\n"
    q_prompt = q_prompt+"""</post>\n"""
q_prompt = q_prompt+"""----\n"""

In [24]:
eval_q_idx= np.random.choice(eval_q_id, size=1, replace=False)

In [25]:
target_post="""\n"""
target_post = target_post+"""<target_post>\n"""
target_post = target_post+df.loc[eval_q_idx, 'question'].values[0]+'\n'
target_post = target_post+"""</target_post>\n"""


In [26]:
tot_prompt = sys_promt+user_prompt_front+target_post+user_prompt_back+q_prompt

In [27]:
with open("output.json", "w") as file:
    json.dump(tot_prompt, file)

In [28]:
df.loc[eval_q_idx, :]

,id,question,answer,answer_encode
26,76541602,"<Title>Pyspark : Add, substract hour/minute/se...",Difficulty class : Intermediate,1


In [29]:
df.loc[eval_q_idx, :]

,id,question,answer,answer_encode
26,76541602,"<Title>Pyspark : Add, substract hour/minute/se...",Difficulty class : Intermediate,1


In [30]:
user_prompt = user_prompt_front+target_post+user_prompt_back+q_prompt

In [31]:
# Example OpenAI Python library request
MODEL = "gpt-4o"
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        # {"role": "system", "content": sys_promt},
        {"role": "user", "content": tot_prompt},
    ],
    temperature=0,
)

PermissionDeniedError: Error code: 403 - {'error': {'message': 'Project `proj_m5SXqRytraFXpjlnKDMPT2Rz` does not have access to model `gpt-4o`', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

In [ ]:
print(json.dumps(json.loads(response.model_dump_json()), indent=4))

In [ ]:
tot_prompt